In [12]:
import os
import sys
import importlib

# Thêm đường dẫn cho các module
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

# Import các module
import import_default
import import_database  
import import_other
import clean_data
import candle_chart

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(clean_data)
importlib.reload(candle_chart)

from import_default import *
from import_database import *
from import_other import *
from clean_data import *
from candle_chart import *

In [13]:
market_period_dict = {
    "period_1": {
        "name": "Sóng hồi phục từ đáy COVID-19",
        "start_date": "2020-04-01",
        "end_date": "2020-07-29",
        "trend": "uptrend",
        "phase": 1
    },
    "period_2": {
        "name": "Sóng tăng vượt đỉnh tiền COVID-19",
        "start_date": "2020-07-29",
        "end_date": "2021-01-29",
        "trend": "uptrend",
        "phase": 2
    },
    "period_3": {
        "name": "Sóng tăng thứ hai sau COVID-19",
        "start_date": "2021-01-29",
        "end_date": "2021-07-20",
        "trend": "uptrend",
        "phase": 3
    },
    "period_4": {
        "name": "Sóng tăng lên đỉnh lịch sử 1500 điểm",
        "start_date": "2021-07-20",
        "end_date": "2022-04-04",
        "trend": "uptrend",
        "phase": 4
    },
    "period_5": {
        "name": "Nhịp điều chỉnh từ đỉnh 1500 điểm",
        "start_date": "2022-04-04",
        "end_date": "2022-07-11",
        "trend": "downtrend",
        "phase": 1
    },
    "period_6": {
        "name": "Sóng hồi trong xu hướng giảm 2022",
        "start_date": "2022-07-11",
        "end_date": "2022-09-05",
        "trend": "downtrend",
        "phase": 2
    },
    "period_7": {
        "name": "Nhịp giảm tạo đáy xu hướng 2022",
        "start_date": "2022-09-05",
        "end_date": "2022-11-16",
        "trend": "downtrend",
        "phase": 3
    },
    "period_8": {
        "name": "Sóng tăng mạnh từ đáy 873 điểm",
        "start_date": "2022-11-16",
        "end_date": "2022-12-06",
        "trend": "uptrend",
        "phase": 1
    },
    "period_9": {
        "name": "Giai đoạn tích lũy sau phục hồi",
        "start_date": "2022-12-06",
        "end_date": "2023-04-26",
        "trend": "sideways",
        "phase": 1
    },
    "period_10": {
        "name": "Sóng tăng chính năm 2023",
        "start_date": "2023-04-26",
        "end_date": "2023-09-12",
        "trend": "uptrend",
        "phase": 1
    },
    "period_11": {
        "name": "Nhịp điều chỉnh mạnh cuối năm 2023",
        "start_date": "2023-09-12",
        "end_date": "2023-11-01",
        "trend": "downtrend",
        "phase": 1
    },
    "period_12": {
        "name": "Sóng tăng cuối 2023 - đầu 2024",
        "start_date": "2023-11-01",
        "end_date": "2024-03-08",
        "trend": "uptrend",
        "phase": 1
    },
    "period_13": {
        "name": "Giai đoạn Sideway kéo dài (2024)",
        "start_date": "2023-11-01",
        "end_date": "2025-01-13",
        "trend": "sideways",
        "phase": 1
    },
    "period_14": {
        "name": "Sóng tăng đầu 2025 (trước thông tin thuế)",
        "start_date": "2025-01-13",
        "end_date": "2025-03-25",
        "trend": "uptrend",
        "phase": 1
    },
    "period_15": {
        "name": "Giảm mạnh do thông tin thuế",
        "start_date": "2025-03-25",
        "end_date": "2025-04-09",
        "trend": "downtrend",
        "phase": 1
    },
    "period_16": {
        "name": "Sóng phục hồi sau thông tin thuế",
        "start_date": "2025-04-09",
        "end_date": "2025-06-10",
        "trend": "uptrend",
        "phase": 1
    },
    "period_17": {
        "name": "Sóng tăng vượt đỉnh",
        "start_date": "2025-06-10",
        "end_date": "2025-07-23",
        "trend": "uptrend",
        "phase": 2
    }
}


#### Đọc dữ liệu từ Mongo

In [14]:
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')
current_quarter_classification_df = get_mongo_collection(ref_db, 'current_quarter_classification')

#### Dữ liệu PTCB - BCTC - CURRENT từ FiinProX

- Đọc và lọc sơ bộ dữ liệu

In [15]:
raw_df_dict = {}
raw_df_dict['ptcb'] = pd.read_excel('../data/raw/raw_ptcb.xlsx', skiprows=7, skipfooter=11)
raw_df_dict['bctc'] = pd.read_excel('../data/raw/raw_bctc.xlsx', skiprows=7, skipfooter=11)
raw_df_dict['current'] = pd.read_excel('../data/raw/raw_current.xlsx', skiprows=7, skipfooter=11)

for key, df in raw_df_dict.items():
    temp_df = df[df.iloc[:, 0].isin(full_stock_classification_df['ticker'].tolist())].reset_index(drop=True)
    raw_df_dict[key] = temp_df

# Tìm ra danh sách cổ phiếu chung cho cả 3 bảng
common_tickers = set(raw_df_dict['ptcb'].iloc[:, 0])
for key, df in raw_df_dict.items():
    common_tickers = common_tickers.intersection(set(df.iloc[:, 0]))

# Lọc tất cả các DataFrame để chỉ bao gồm các mã cổ phiếu chung
for key, df in raw_df_dict.items():
    raw_df_dict[key] = df[df.iloc[:, 0].isin(common_tickers)].reset_index(drop=True)

- Làm sạch dữ liệu

In [16]:
ptcb_name_map = {
    'ROE % (TTM)': 'ROE',
    'ROA % (TTM)': 'ROA',
    'Tỷ suất lợi nhuận thuần (TTM)': 'Tỷ suất lợi nhuận thuần',
    'Tăng trưởng doanh thu thuần (YoY) (Quý)': 'Tăng trưởng doanh thu thuần',
    'Tăng trưởng lợi nhuận thuần (YoY) (Quý)': 'Tăng trưởng lợi nhuận thuần',
    'Tổng công nợ/ Vốn chủ sở hữu (TTM)': 'Tổng nợ/VCSH',
    'Tổng vốn vay/ VSCH (TTM)': 'Tổng vốn vay/VCSH',
    'Tổng vốn vay/ Tổng TS (TTM)': 'Tổng vốn vay/Tổng TS',
    'Tỷ lệ thanh toán hiện thời (TTM)': 'Tỷ lệ thanh toán hiện thời',
    '% Doanh thu kế hoạch (TTM)': 'Tỷ lệ hoàn thành doanh thu',
    '% LNST kế hoạch (TTM)': 'Tỷ lệ hoàn thành LNST',
    'Tỷ suất cổ tức (TTM)': 'Tỷ suất cổ tức',
    'Tỷ suất thanh toán cổ tức (TTM)': 'Tỷ suất chi trả cổ tức',
    'P/E cơ bản (TTM)': 'P/E',
    'P/B (TTM)': 'P/B',
    'Giá trị doanh nghiệp/ EBITDA (TTM)': 'EV/EBITDA',
    'Vốn hóa thị trường (TTM)': 'Vốn hóa thị trường',

    # ===== Các chỉ số mới cho ngành Ngân hàng =====
    'CASA (TTM)': 'CASA',
    'NIM (TTM)': 'NIM',
    'Tỷ lệ Chi phí/ Thu nhập (TTM)': 'CIR',
    'ROA trước dự phòng % (TTM)': 'ROA trước dự phòng',
    'Nợ xấu (3-5)/ Tổng dư nợ (TTM)': 'Tỷ lệ nợ xấu',
    'Trích lập dự phòng/ Nợ xấu (TTM)': 'Tỷ lệ bao phủ nợ xấu',
    'Tăng trưởng tín dụng (TTM)': 'Tăng trưởng tín dụng',
    'Tăng trưởng tiền gửi (TTM)': 'Tăng trưởng tiền gửi',
    'Dư nợ cho vay/Vốn chủ sở hữu (TTM)': 'Dư nợ cho vay/VCSH',
    'Dư nợ cho vay/ Tiền gửi (TTM)': 'LDR'
}

ptcb_group_map = {
    # --- Các nhóm cho doanh nghiệp thông thường ---
    'Hiệu quả': [
        'ROE',
        'ROA',
        'Tỷ suất lợi nhuận thuần'
    ],
    'Tăng trưởng': [
        'Tăng trưởng doanh thu thuần',
        'Tăng trưởng lợi nhuận thuần'
    ],
    'Sức khỏe tài chính': [
        'Tổng nợ/VCSH',
        'Tổng vốn vay/VCSH',
        'Tổng vốn vay/Tổng TS',
        'Tỷ lệ thanh toán hiện thời'
    ],
    'So với kế hoạch': [
        'Tỷ lệ hoàn thành doanh thu',
        'Tỷ lệ hoàn thành LNST'
    ],
    'Cổ tức': [
        'Tỷ suất cổ tức',
        'Tỷ suất chi trả cổ tức'
    ],
    'Định giá': [
        'P/E',
        'P/B',
        'EV/EBITDA',
        'Vốn hóa thị trường'
    ],
    
    # ===== NHÓM RIÊNG CHO NGÀNH NGÂN HÀNG =====
    'Ngân hàng': {
        'CASA',
        'NIM',
        'CIR',
        'ROA trước dự phòng',
        'Tăng trưởng tín dụng',
        'Tăng trưởng tiền gửi', 
        'Tỷ lệ nợ xấu',
        'Tỷ lệ bao phủ nợ xấu',
        'Dư nợ cho vay/VCSH',
        'LDR'
    }
}
ptcb_group_map = {metric: group for group, metrics in ptcb_group_map.items() for metric in metrics}

In [17]:
current_name_map = {
    'Giá đóng cửa gần nhất': 'Giá đóng cửa',
    'Khối lượng trung bình 1 tuần': 'KLGD TB 1 tuần',
    'Khối lượng trung bình 1 tháng': 'KLGD TB 1 tháng',
    'Giá trị trung bình 1 tuần': 'GTGD TB 1 tuần',
    'Giá trị trung bình 1 tháng': 'GTGD TB 1 tháng',
    'Số CP lưu hành hiện thời': 'CP đang lưu hành',
    'Free Float': 'Free Float',
    '% Free Float': '% Free Float',
    'Khối lượng khối ngoại sở hữu tối đa': 'KLNN sở hữu tối đa',
    'Tỷ lệ sở hữu khối ngoại': 'Tỷ lệ sở hữu NN',
    'Khối lượng khối ngoại còn lại': 'KLNN còn lại',
    'Room khối ngoại còn lại': 'Room NN còn lại',
    'Tỷ lệ sở hữu nhà nước': 'Tỷ lệ sở hữu nhà nước',
    'EPS cơ bản ': 'EPS',
    'P/E cơ bản ': 'P/E',
    'P/B ': 'P/B',
    'Giá trị doanh nghiệp/ EBITDA ': 'EV/EBITDA',
    'Vốn hóa thị trường ': 'Vốn hóa thị trường',
    'Giá trị ròng khối ngoại': 'NN mua ròng (GT)',
    'Giá trị ròng tự doanh': 'Tự doanh mua ròng (GT)'
}

current_group_map = {
    'Thanh khoản & Giá': [
        'Giá đóng cửa',
        'KLGD TB 1 tuần',
        'KLGD TB 1 tháng',
        'GTGD TB 1 tuần',
        'GTGD TB 1 tháng'
    ],
    'Cơ cấu cổ đông': [
        'CP đang lưu hành',
        'Free Float',
        '% Free Float',
        'KLNN sở hữu tối đa',
        'Tỷ lệ sở hữu NN',
        'KLNN còn lại',
        'Room NN còn lại',
        'Tỷ lệ sở hữu nhà nước'
    ],
    'Định giá & Hiệu suất': [
        'EPS',
        'P/E',
        'P/B',
        'EV/EBITDA',
        'Vốn hóa thị trường'
    ],
    'Dòng tiền tổ chức': [
        'NN mua ròng (GT)',
        'Tự doanh mua ròng (GT)'
    ]
}
current_group_map = {metric: group for group, metrics in current_group_map.items() for metric in metrics}


In [18]:
# 1. Bảng ánh xạ tên đầy đủ và tên viết tắt
bctc_name_map = {
    'I. TÀI SẢN NGẮN HẠN': 'Tài sản ngắn hạn',
    '1. Tiền và các khoản tương đương tiền': 'Tiền & tương đương tiền',
    '4. Hàng tồn kho': 'Hàng tồn kho',
    'II. TÀI SẢN DÀI HẠN': 'Tài sản dài hạn',
    '2. Tài sản cố định': 'Tài sản cố định',
    'I. NỢ PHẢI TRẢ': 'Nợ phải trả',
    '1. Nợ ngắn hạn': 'Nợ ngắn hạn',
    '2. Nợ dài hạn': 'Nợ dài hạn',
    'II. VỐN CHỦ SỞ HỮU': 'Vốn chủ sở hữu',
    '3. Doanh thu thuần bán hàng và cung cấp dịch vụ': 'Doanh thu thuần',
    '4. Giá vốn hàng bán': 'Giá vốn hàng bán',
    '5. Lợi nhuận gộp về bán hàng và cung cấp dịch vụ': 'Lợi nhuận gộp',
    '7. Chi phí tài chính': 'Chi phí tài chính',
    '7.1. Trong đó: Chi phí lãi vay': 'Chi phí lãi vay',
    '9. Chi phí bán hàng': 'Chi phí bán hàng',
    '11. Lợi nhuận thuần từ hoạt động kinh doanh': 'Lợi nhuận thuần từ HĐKD',
    '18. Lợi nhuận sau thuế TNDN': 'Lợi nhuận sau thuế',
    'I. Lưu chuyển tiền thuần từ hoạt động kinh doanh': 'LC tiền từ HĐKD',
    'II. Lưu chuyển tiền thuần từ hoạt động đầu tư': 'LC tiền từ HĐĐT',
    'III. Lưu chuyển tiền thuần từ hoạt động tài chính': 'LC tiền từ HĐTC',
    '3. Tiền gửi của khách hàng': 'Tiền gửi của khách hàng',
    '6.1. Cho vay khách hàng': 'Cho vay khách hàng',
    '6. Phát hành giấy tờ có giá': 'Phát hành giấy tờ có giá',
    '1. Thu nhập lãi thuần': 'Thu nhập lãi thuần',
    '2. Lãi/ lỗ thuần từ hoạt động dịch vụ': 'Lãi/lỗ thuần từ dịch vụ',
    '9. Chi phí hoạt động': 'Chi phí hoạt động',
    '11. Chi phí dự phòng rủi ro tín dụng': 'Chi phí dự phòng RRTD',
    '10. Lợi nhuận thuần từ hoạt động kinh doanh trước chi phí dự phòng rủi ro tín dụng': 'LN thuần trước dự phòng',
    '1.2.1. Các tài sản tài chính ghi nhận thông qua lãi lỗ (FVTPL)': 'Tài sản TC FVTPL',
    '1.2.2. Các khoản đầu tư nắm giữ đến ngày đáo hạn (HTM)': 'Đầu tư HTM',
    '1.2.4. Các khoản tài chính sẵn sàng để bán (AFS)': 'Tài sản TC AFS',
    '1.1. Lãi từ các tài sản tài chính ghi nhận thông qua lãi/lỗ ( FVTPL)': 'Lãi từ FVTPL',
    '1.2. Lãi từ các khoản đầu tư nắm giữ đến ngày đáo hạn (HTM)': 'Lãi từ HTM',
    '1.4. Lãi từ các tài sản tài chính sẵn sàng để bán (AFS)': 'Lãi từ AFS',
    '4.1. Lỗ các tài sản tài chính ghi nhận thông qua lãi lỗ (FVTPL)': 'Lỗ từ FVTPL',
    '4.2. Lỗ các khoản đầu tư nắm giữ đến ngày đáo hạn (HTM)': 'Lỗ từ HTM',
    '4.4. Lỗ và ghi nhận chênh lệch đánh giá theo giá trị hợp lý tài sản tài chính sẵn sàng để bán (AFS) khi phân loại lại': 'Lỗ từ AFS',
    '1.2.3. Các khoản cho vay': 'Các khoản cho vay',
    '1.6. Doanh thu hoạt động môi giới chứng khoán': 'DT từ môi giới CK',
    '1.3. Lãi từ các khoản cho vay và phải thu': 'Lãi từ cho vay',
    '1.9. Doanh thu hoạt động tư vấn đầu tư chứng khoán': 'DT từ tư vấn đầu tư CK'
}

# 2. Bảng phân nhóm với cấu trúc key là TÊN NHÓM, phân loại theo BCTC và ngành
bctc_group_map_original = {
    'CĐKT - Tài sản': [
        'Tài sản ngắn hạn', 'Tiền & tương đương tiền', 'Hàng tồn kho',
        'Tài sản dài hạn', 'Tài sản cố định'
    ],
    'CĐKT - Nguồn vốn': [
        'Nợ phải trả', 'Nợ ngắn hạn', 'Nợ dài hạn', 'Vốn chủ sở hữu'
    ],
    'KQKD - Doanh nghiệp': [
        'Doanh thu thuần', 'Giá vốn hàng bán', 'Lợi nhuận gộp', 'Chi phí tài chính',
        'Chi phí lãi vay', 'Chi phí bán hàng', 'Lợi nhuận thuần từ HĐKD', 'Lợi nhuận sau thuế'
    ],
    'LCTT - Dòng tiền': [
        'LC tiền từ HĐKD', 'LC tiền từ HĐĐT', 'LC tiền từ HĐTC'
    ],
    'CĐKT - Ngân hàng': [
        'Tiền gửi của khách hàng', 'Cho vay khách hàng', 'Phát hành giấy tờ có giá'
    ],
    'KQKD - Ngân hàng': [
        'Thu nhập lãi thuần', 'Lãi/lỗ thuần từ dịch vụ', 'Chi phí hoạt động',
        'Chi phí dự phòng RRTD', 'LN thuần trước dự phòng'
    ],
    'CĐKT - CTCK': [
        'Tài sản TC FVTPL', 'Đầu tư HTM', 'Tài sản TC AFS', 'Các khoản cho vay',
        
    ],
    'KQKD - CTCK': [
        'Lãi từ cho vay', 'DT từ môi giới CK', 'DT từ tư vấn đầu tư CK', 'Lãi từ FVTPL', 'Lãi từ HTM', 'Lãi từ AFS', 'Lỗ từ FVTPL', 'Lỗ từ HTM', 'Lỗ từ AFS'
    ]
}

# Chuyển đổi từ {nhóm: [chỉ số]} thành {chỉ số: nhóm} để dễ tra cứu
bctc_group_map = {metric: group for group, metrics in bctc_group_map_original.items() for metric in metrics}

In [19]:
clean_df_dict = {}
for key, df in raw_df_dict.items():
    temp_df = transform_to_long_format(df)
    temp_df.to_csv(f'../data/clean/clean_{key}.csv', index=False)
    if key == 'current':
        foreign_buy_cols = [
            'Giá trị khớp lệnh trung bình khối ngoại mua 1 tuần',
            'Giá trị khớp lệnh trung bình khối ngoại mua 1 tháng', 
            'Giá trị thỏa thuận trung bình khối ngoại mua 1 tuần',
            'Giá trị thỏa thuận trung bình khối ngoại mua 1 tháng'
        ]

        foreign_sell_cols = [
            'Giá trị khớp lệnh trung bình khối ngoại bán 1 tuần',
            'Giá trị thỏa thuận trung bình khối ngoại bán 1 tuần',
            'Giá trị khớp lệnh trung bình khối ngoại bán 1 tháng', 
            'Giá trị thỏa thuận trung bình khối ngoại bán 1 tháng'
        ]

        prop_buy_cols = [
            'Giá trị khớp lệnh trung bình tự doanh mua 1 tuần',
            'Giá trị thỏa thuận trung bình tự doanh mua 1 tuần',
            'Giá trị khớp lệnh trung bình tự doanh mua 1 tháng',
            'Giá trị thỏa thuận trung bình tự doanh mua 1 tháng'
        ]

        prop_sell_cols = [
            'Giá trị khớp lệnh trung bình tự doanh bán 1 tuần',
            'Giá trị thỏa thuận trung bình tự doanh bán 1 tuần', 
            'Giá trị khớp lệnh trung bình tự doanh bán 1 tháng',
            'Giá trị thỏa thuận trung bình tự doanh bán 1 tháng'
        ]

        all_trading_cols = foreign_buy_cols + foreign_sell_cols + prop_buy_cols + prop_sell_cols

        # Tính net value cho từng ticker
        foreign_net = temp_df[temp_df['name'].isin(foreign_buy_cols)].groupby('ticker')['value'].sum() - \
                    temp_df[temp_df['name'].isin(foreign_sell_cols)].groupby('ticker')['value'].sum()

        prop_net = temp_df[temp_df['name'].isin(prop_buy_cols)].groupby('ticker')['value'].sum() - \
                temp_df[temp_df['name'].isin(prop_sell_cols)].groupby('ticker')['value'].sum()

        # Lấy thông tin ticker từ DataFrame gốc để copy các cột khác (chỉ lấy 1 record đầu tiên cho mỗi ticker)
        ticker_info = temp_df[['ticker', 'industry', 'period', 'unit']].drop_duplicates(subset=['ticker']).set_index('ticker')

        # Tạo DataFrame mới với đầy đủ các cột
        foreign_net_df = foreign_net.reset_index()
        foreign_net_df['name'] = 'Giá trị ròng khối ngoại'
        foreign_net_df['industry'] = foreign_net_df['ticker'].map(ticker_info['industry'])
        foreign_net_df['period'] = foreign_net_df['ticker'].map(ticker_info['period'])
        foreign_net_df['unit'] = 'Triệu VND'  # Đặt unit phù hợp cho giá trị ròng

        prop_net_df = prop_net.reset_index() 
        prop_net_df['name'] = 'Giá trị ròng tự doanh'
        prop_net_df['industry'] = prop_net_df['ticker'].map(ticker_info['industry'])
        prop_net_df['period'] = prop_net_df['ticker'].map(ticker_info['period'])
        prop_net_df['unit'] = 'Triệu VND'  # Đặt unit phù hợp cho giá trị ròng

        # Cập nhật clean_df_dict
        temp_df = pd.concat([
            temp_df[~temp_df['name'].isin(all_trading_cols)],
            foreign_net_df,
            prop_net_df
        ], ignore_index=True)

    elif key == 'ptcb':
        temp_df['name'] = temp_df['name'].map(ptcb_name_map)
        temp_df['group'] = temp_df['name'].map(ptcb_group_map)

    elif key == 'bctc':
        temp_df['name'] = temp_df['name'].map(bctc_name_map)
        temp_df['group'] = temp_df['name'].map(bctc_group_map)

    clean_df_dict[key] = temp_df

#### BCTC trong 4 năm gần nhất

In [20]:
# fs_dict = get_financial_statements(
#     stock=main_stock,
#     year='2025', 
#     quarter='2',
#     period_count='20'
# )

# fs_df = pd.DataFrame()
# for report_type in ['BalanceSheet', 'IncomeStatement', 'CashFlow']:
#     temp_df = fs_dict[report_type]
#     temp_df['type'] = report_type
#     fs_df = pd.concat([fs_df, temp_df], ignore_index=True)

# fs_df.to_csv(f'../output/csv/fs_stock.csv', index=False, encoding='utf-8-sig')

#### Lưu các dữ liệu để phân tích

In [22]:
industry_stock_list = ["SSI", "VIX", "VND", "VDS", "HCM", "SHS", "MBS", "CTS", "ORS", "VCI", "FTS", "BSI", "BVS"]

current_df = clean_df_dict['current']
current_df = current_df[current_df['ticker'].isin(industry_stock_list)]
current_df.to_csv('../output/clean_current.csv', index=False, encoding='utf-8-sig')

ptcb_df = clean_df_dict['ptcb']
ptcb_df = ptcb_df[ptcb_df['ticker'].isin(industry_stock_list)]
ptcb_df.to_csv('../output/clean_ptcb.csv', index=False, encoding='utf-8-sig')

bctc_df = clean_df_dict['bctc']
bctc_df = bctc_df[bctc_df['ticker'].isin(industry_stock_list)]
bctc_df = bctc_df[~bctc_df['group'].str.contains('CTCK', na=False)]
bctc_df = bctc_df[~bctc_df['group'].str.contains('CTCK', na=False)]
bctc_df.to_csv('../output/clean_bctc.csv', index=False, encoding='utf-8-sig')